# COMPAS Tutorial Run (Python)

## Downloading COMPAS to your laptop

To download COMPAS, read the "Getting Started" on COMPAS.readthedocs
https://compas.readthedocs.io/en/latest/index.html

Getting Started: Getting your environment ready to simulate a population.


## Running COMPAS via Python: 
Simulate your population using python and the command line.

All the basic packages and script you need are in 

path-to-compas-folder/COMPAS/compas_python_utils/preprocessing

The preprocessing folder will have:
1. **runSubmit.py**: when executed, will generate your population.
2. **compasConfigDefault.yaml**: will have all the parameters that you can customize for your population.
3. any **grid.txt**: an optional way of customizing your populations even further.
4. **COMPAS_Output folders**: the folder that will contain the outputs of your population and run details of your simulation.

#### Step 1: cd to the preprocessing folder

In your command line, cd path-to-compas-folder/COMPAS/compas_python_utils/preprocessing

If you are not sure of where your COMPAS folder is located in your computer:
1. Search for your folder with your computer's "Finder".
2. Open your folder info.
3. Your folder will have its filepath.

#### Step 2: Understanding the .yaml file
The 'compasConfigDefault.yaml' has all the input parameters about the binary systems you are about to simulate.

**Some of the parameter values with a default .yaml file:**\
Detailed Output = False\
Number of binary systems = 10 binaries\
Metallicity = 0.014200 (Solar Metallicity)\
Eccentricity = 0.0\
Minimum initial mass for primary star = 5 (M_sol)\
Maximum initial mass for primary star = 150 (M_sol)\
Minimum initial mass for secondary star = 0.1 (M_sol)

$\Rightarrow$ Based on the project your project, these parameters should be changed to get the desired outputs in your simulation.

#### Running a simulation

**At this point you should have:**
1. installed COMPAS from GitHub
2. installed the necessary python packages (numpy, matplotlib, h5py, pandas)
3. and moved into the preprocessing folder within COMPAS

**Now let's change some of the contents in the .yaml folder:**
1. open the .yaml folder.
2. uncomment line 88: # --number-of-systems: 10
3. and change line 88 to say: --number-of-systems: 1000
by changing "--number-of-systems: 1000", you want 1000 binary stellar evolutions.
4. uncomment line 99: # --metallicity: 0.014200 (this is solar metallicity)
5. and change line 99 to say: --metallicity: 0.00142
by changing "--metallicity: 0.00142", you want all 1000 binary stellar evolutions to have ~1/10 solar metallicity
6. save your changes

**Now type this into your command line:**\
python3 runSubmit.py

**Your output should look something like...**

Start generating binaries at Wed Jun 18 10:40:58 2025

0: Stars merged: (Main_Sequence_>_0.7 -> Main_Sequence_>_0.7) + (Main_Sequence_>_0.7 -> Main_Sequence_>_0.7)\
1: Allowed time exceeded: (Main_Sequence_>_0.7 -> Oxygen-Neon_White_Dwarf) + (Main_Sequence_<=_0.7 -> Main_Sequence_<=_0.7)\
2: Double White Dwarf formed: (Main_Sequence_>_0.7 -> Oxygen-Neon_White_Dwarf) + (Main_Sequence_>_0.7 -> Carbon-Oxygen_White_Dwarf)\
3: Allowed time exceeded: (Main_Sequence_>_0.7 -> Oxygen-Neon_White_Dwarf) + (Main_Sequence_<=_0.7 -> Main_Sequence_<=_0.7)\
4: Stars merged: (Main_Sequence_>_0.7 -> First_Giant_Branch) + (Main_Sequence_>_0.7 -> Main_Sequence_>_0.7)\
5: Double White Dwarf formed: (Main_Sequence_>_0.7 -> Carbon-Oxygen_White_Dwarf) + (Main_Sequence_>_0.7 -> Carbon-Oxygen_White_Dwarf)

...

**and so on until this message at the end.**

Generated 1000 of 1000 binaries requested

Simulation completed

End generating binaries at Wed Jun 18 10:43:56 2025

Clock time = 177.109 CPU seconds
Wall time  = 0000:00:35 (hhhh:mm:ss)

#### Accessing output data with Python
Your COMPAS output will be within "preprocessing", the same folder as the .yaml file.

Now that you ran a simulation, you can access the data with python! Run the cells in the notebook to create your first plot with your data!

**I added a #complete to each area that needs completing by you**

In [21]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt

In [ ]:
"""==========================================
The h5.File opens your hdf5 COMPAS output.

This line assigns your opened file to an
object that I call 'data'.
=========================================="""

# Add your unique filepath to the preprocessing folder
data = h5.File("/path-to-folder/COMPAS_Output_1/COMPAS_Output.h5") #complete

In [ ]:
"""=================================================================
 The hdf5 file is large! It contains lots of data on the stars you 
 just evolved. 

 You can run this code and view how the data is organized within a 
 COMPAS output file. 

 I also added a section to assign each 'chapter' of your data so that 
 you can see the subsections as well!
===================================================================="""

print("The 'chapters' within your book of data:")
print(data.keys())

# Assign all keys to an object
CEs = data['BSE_Common_Envelopes']
DCOs = data['BSE_Double_Compact_Objects']
RLOFs = data['BSE_RLOF']
SNs = data['BSE_Supernovae']
SPs = data['BSE_System_Parameters']
Run_Details = data['Run_Details']
print("An example of the subsections within one key of your COMPAS data:")
print(Run_Details.keys())

In [ ]:
"""=================================================================
 We can plot are the initial and final masses 
 of the stars that became double compact object mergers.

 Run this cell to initialize the definition I made for isolating
 double compact object mergers.
===================================================================="""

def dco_masses(h5_data_file):
    """
    Returns only the initial and final masses for BSE systems that became DCO mergers.
    ==============================================================================

    Inputs:
    --------
    h5_data_file: h5 File
        The hdf5 file of your simulation

    Outputs:
    --------
    zams_mass1: array-like
        The initial mass of primary star.

    zams_mass2: array-like
        The initial mass of secondary star.

    co_mass1: array-like
        The final mass of primary star.
    
    co_mass2: array-like
        The final mass of secondary star.
    """

    # Extract datasets
    dco_seed = h5_data_file['BSE_Double_Compact_Objects']['SEED'][:]  # DCO seeds
    system_seed = h5_data_file['BSE_System_Parameters']['SEED'][:]     # All seeds
    ce_all_seed = np.unique(h5_data_file['BSE_RLOF']['SEED'][:]) # Common Envelope seeds
    ce_seed = np.array([seed for seed in ce_all_seed if seed in dco_seed])
    # BSEs that experience common envelope will often have more than one event, so get rid of repeating seeds.
    full_zams_mass1 = h5_data_file['BSE_System_Parameters']['Mass@ZAMS(1)'][:]
    full_zams_mass2 = h5_data_file['BSE_System_Parameters']['Mass@ZAMS(2)'][:]
    full_co_mass1 = h5_data_file['BSE_Double_Compact_Objects']['Mass(1)'][:]
    full_co_mass2 = h5_data_file['BSE_Double_Compact_Objects']['Mass(2)'][:]

    zams1_dict = dict(zip(system_seed, full_zams_mass1))
    zams2_dict = dict(zip(system_seed, full_zams_mass2))
    
    # zams and co masses
    zams_mass1 = np.array([zams1_dict[seed] for seed in dco_seed])
    zams_mass2 = np.array([zams2_dict[seed] for seed in dco_seed])

    co_mass1 = full_co_mass1
    co_mass2 = full_co_mass2

    return zams_mass1, zams_mass2, co_mass1, co_mass2
    

In [ ]:
zams1, zams2, co1, co2 = dco_masses(data)

fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(zams1, co1, marker='*', color='C0', label='Primary Star')
ax.scatter(zams1, co2, marker='*', color='C1', label='Secondary Star')
ax.legend(loc = 'lower right')
ax.set_xlabel(r'Mass at ZAMS, $M_{\odot}$')
ax.set_ylabel(r'Mass as Compact Object, $M_{\odot}$')
ax.set_title("Initial Mass at Zero-Age Main-Sequence (ZAMS) and Final Mass as Double Compace Object (DCO)")
ax.grid()
plt.show()